# Can LLMs Do Math?

## Your First Hands-On Exploration of Large Language Models

Welcome! In this notebook, you'll discover what happens when we ask an LLM to solve math problems. 

**Guiding Question:** Are LLMs *computing* answers, or *predicting* them?

## Setup: Import Libraries and Load the Model

First, we'll import the FAIR-LLM framework and load a small, fast language model.

In [ ]:
import os
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import asyncio
from dotenv import load_dotenv
from fairlib.modules.mal.huggingface_adapter import HuggingFaceAdapter
from fairlib.core.message import Message

# Load your Hugging Face token from .env file
load_dotenv()
token = os.getenv("HUGGING_FACE_HUB_TOKEN")

if not token:
    print("Warning: HUGGING_FACE_HUB_TOKEN not found in .env file!")
    print("Please follow the README instructions to set up your token.")
else:
    print("Token loaded successfully!")

### Load the Language Model

We're using a small Dolphin model (only 3B parameters) that can run on your laptop.

**Note:** The first time you run this, it will download the model and cache it on your device. This will take a while be patient.

In [ ]:
# Load the model
print("Loading language model...")
llm = HuggingFaceAdapter(model_name="dolphin3-qwen25-3b", auth_token=token)
print("Model loaded! Ready to test.")

## Experiment 1: Simple Math (Common Problems)

Let's start with arithmetic that appears MILLIONS of times in training data.

**Hypothesis:** The LLM will succeed because it has "seen" these exact problems before.

In [ ]:
def test_math(question, correct_answer):
    """Helper function to test the LLM on a math problem"""
    messages = [Message(role="user", content=question)]
    response = llm.invoke(messages)
    
    print(f"\nQuestion: {question}")
    print(f"LLM Answer: {response.content}")
    print(f"Correct Answer: {correct_answer}")
    
    # Simple check if the answer contains the right number
    if str(correct_answer) in response.content:
        print("✓ SUCCESS!")
    else:
        print("✗ FAILED!")
    
    return response.content

In [ ]:
# Test 1: Very simple addition
test_math("2 + 2?", 4)

In [ ]:
# Test 2: Simple multiplication
test_math("10 × 10?", 100)

In [ ]:
# Test 3: Basic exponent
test_math("What is 5 squared?", 25)

### Reflection Question 1

**Did the LLM succeed on these simple problems?**

Think about:
- How many times has "2 + 2 = 4" appeared on the internet?
- Is the LLM *computing* this answer, or *remembering* it from training data?

## Experiment 2: Complex Math (Uncommon Problems)

Now let's try calculations that are UNLIKELY to appear in training data.

**Hypothesis:** The LLM will struggle because these specific calculations probably weren't in its training data.

In [ ]:
# Test 4: Uncommon multiplication
test_math("237 × 843?", 199791)

In [ ]:
# Test 5: Larger uncommon multiplication
test_math("8347 × 6291?", 52510977)

In [ ]:
# Test 6: Multi-step calculation
test_math("(456 + 789) × 234?", 291330)

### Reflection Question 2

**What happened with the complex problems?**

Think about:
- Did the LLM get the exact right answer?
- If it was wrong, was it *close*? Or completely off?
- What does this tell you about how the LLM is "solving" these problems?

## Understanding What's Happening

### The LLM's Process:

When you ask "What is 237 × 843?", the LLM:

1. **Tokenizes** the input: `["What", "is", "237", "×", "843", "?"]`
2. **Uses self-attention** to look at all tokens simultaneously
3. **Predicts** the most likely next tokens based on patterns it saw during training
4. **Generates** a sequence of digits that "looks like" an answer

### It's NOT:
- ❌ Applying multiplication algorithms
- ❌ Using a calculator
- ❌ Computing the result

### It IS:
- ✅ Pattern matching from training data
- ✅ Predicting "what numbers usually follow this pattern"
- ✅ Making educated guesses based on statistical patterns

## Experiment 3: Let's Break It Further

Try asking the LLM to explain its reasoning:

In [ ]:
messages = [
    Message(
        role="user",
        content="Solve 8347 × 6291 step by step. Show your work."
    )
]

response = llm.invoke(messages)
print(f"LLM's 'Work':\n{response.content}")
print(f"\nActual answer: {8347 * 6291}")

### Reflection Question 3

**Did the LLM's "step by step" work actually compute the right answer?**

This reveals something profound: The LLM can generate text that *looks like* mathematical reasoning, but it's not actually performing the computation!

## The Key Insight

### LLMs Don't Compute - They Predict!

**Simple problems:** The exact pattern (`2 + 2 = 4`) appeared millions of times in training data
→ LLM *remembers* and succeeds 

**Complex problems:** The exact pattern (`8347 × 6291 = 52507377`) probably never appeared in training data  
→ LLM *guesses* based on "big number × big number = bigger number" pattern  
→ LLM fails 

### This is Pattern Matching, Not Calculation

The LLM is:
- Predicting the next token (number)
- Using statistical patterns from training
- NOT applying mathematical algorithms
- NOT using a calculator


## Your Turn: Experiment!

Try your own math problems. See if you can find the boundary between "succeeds" and "fails":


In [ ]:
your_question = "What is 15 × 23?"  # Change this to whatever you want
your_answer = 15 * 23

test_math(your_question, your_answer)

## So What's Next? Why Do We Need Agents?

You've just discovered a fundamental limitation of LLMs:
- **They're amazing at language understanding and generation**
- **But they can't reliably execute tasks that require computation**

### The Solution: LLM Agents

What if we could:
1. Keep the LLM's language understanding ("the user wants to multiply two numbers")
2. Give it access to actual TOOLS (a real calculator, Python code, databases, APIs)
3. Let it decide WHEN to use tools vs. when to use its own knowledge

**That's what we're going to build!**

### Agent Architecture:
```
User: "What's 8347 × 6291?"
  ↓
LLM: "I need to multiply. I should use the calculator tool."
  ↓
Tool: calculator(8347, 6291) → 52507377
  ↓
LLM: "The answer is 52,507,377"
  ↓
User: Correct!
```

**LLM = Understanding**  
**Tools = Execution**  
**Agent = LLM + Tools = Actually useful!**

## Building Your First Agent: Calculator Edition

Now let's put everything together! We'll build an agent that:
1. **Understands** your math question (LLM brain)
2. **Decides** when to use a calculator (reasoning)
3. **Executes** the calculation (tool use)
4. **Responds** with the correct answer

This is the bridge from "predicting text" to "actually computing"!

### Step 1: Import the FAIR-LLM Components

We need to import the building blocks for our agent:
- **HuggingFaceAdapter**: The LLM "brain"
- **ToolRegistry**: Holds all available tools
- **SafeCalculatorTool**: A calculator the agent can use
- **ToolExecutor**: Actually runs the tools
- **WorkingMemory**: Keeps track of the conversation
- **SimpleAgent**: Brings everything together
- **SimpleReActPlanner**: The reasoning engine (ReAct = Reason + Act)
- **RoleDefinition**: Defines the agent's persona

In [ ]:
from fairlib import (
    HuggingFaceAdapter,
    ToolRegistry,
    SafeCalculatorTool,
    ToolExecutor,
    WorkingMemory,
    SimpleAgent, 
    SimpleReActPlanner,
    RoleDefinition
)

print("All components imported!")

### Step 2: Create the Agent's "Brain" (LLM)

We'll reuse the same LLM from earlier - but now instead of asking it to do math directly, we'll TEACH IT to USE TOOLS.

In [ ]:
# The same LLM, but now it will be part of an agent system
agent_llm = HuggingFaceAdapter(model_name="dolphin3-qwen25-3b", auth_token=token)
print("Agent brain created!")

### Step 3: Give the Agent a Tool (Calculator)

This is the KEY difference! Instead of making the LLM do math by prediction, we give it access to a REAL calculator tool.

In [ ]:
# Create a registry to hold all tools
tool_registry = ToolRegistry()

# Create the calculator tool
calculator_tool = SafeCalculatorTool()

# Register it so the agent knows it exists
tool_registry.register_tool(calculator_tool)

# See what tools are available
available_tools = [tool.name for tool in tool_registry.get_all_tools().values()]
print(f"Agent's tools: {available_tools}")

### Step 4: Create the Tool Executor

The executor is like the agent's "hands" - it actually runs the tools when the agent decides to use them.

In [ ]:
# The executor can run any tool in the registry
executor = ToolExecutor(tool_registry)

print("Tool executor created!")

### Step 5: Give the Agent Memory

The agent needs memory to keep track of the conversation and what it has already tried.

In [ ]:
# Simple working memory for the conversation
memory = WorkingMemory()

print("Memory system created!")

### Step 6: Create the Planner (The Reasoning Engine)

This is where the magic happens! The planner uses the **ReAct** pattern:
- **Reason**: Think about what to do next
- **Act**: Take an action (use a tool or respond)

The planner looks at the conversation, decides if it needs to use a tool, and calls it if needed.

In [ ]:
# Create the planner
planner = SimpleReActPlanner(agent_llm, tool_registry)

# CRITICAL: Properly orient the agent with detailed instructions
planner.prompt_builder.role_definition = RoleDefinition(
    "You are an expert mathematical calculator. Your job is to perform mathematical calculations. "
    "You reason step-by-step to determine the best course of action. "
    "When you receive a calculation request, you MUST use the calculator tool - do not attempt to calculate in your head. "
    "If a user's request requires multiple steps, you must break it down and execute them sequentially. "
    "You must follow the strict formatting rules for tool calling. "
    "Always use tools for calculations to ensure accuracy."
)

print("Planner created with detailed role definition!")

### Step 7: Assemble the Complete Agent

Now we bring all the pieces together into a functioning agent!

In [ ]:
# Create the agent with all components
agent = SimpleAgent(
    llm=agent_llm,
    planner=planner,
    tool_executor=executor,
    memory=memory,
    max_steps=10
)

print("Agent successfully created!")
print("The agent can now:")
print("  Understand your math questions")
print("  Decide when to use the calculator")
print("  Execute calculations accurately")
print("  Respond with correct answers")

## Testing the Agent

Let's test our agent on the SAME problems that the pure LLM failed on!

In [ ]:
# Test function for the agent
async def test_agent_math(question):
    print(f"\nQuestion: {question}")
    print("Agent thinking...")
    
    response = await agent.arun(question) # this line invokes the entire agent reasoning loop!
    
    print(f"Agent: {response}\n")
    return response

### Test 1: Simple Math (This Should Still Work)

In [ ]:
# Test on simple problem
await test_agent_math("What is 2 + 2?")

### Test 2: Complex Math (The LLM Failed This - Will the Agent Succeed?)

In [ ]:
# The problem the LLM got wrong!
await test_agent_math("What is 8347 × 6291?")

### Test 3: Multi-Step Calculation

In [ ]:
await test_agent_math("What is (456 + 789) × 234?")

## 🎯 The Key Insight: What Changed?

### Pure LLM (Earlier):
```
User: "What is 8347 × 6291?"
  ↓
LLM: [predicts next tokens based on patterns]
  ↓
LLM: "52,487,000" (wrong - it guessed!)
```

### LLM Agent (Now):
```
User: "What is 8347 × 6291?"
  ↓
Agent: [reasons] "I need to calculate. I should use the calculator tool."
  ↓
Agent: [calls] calculator(8347, 6291)
  ↓
Calculator: 52,507,377
  ↓
Agent: "The answer is 52,507,377"
```

**The LLM didn't get better at math - we gave it the RIGHT TOOL for the job!**

## Final Reflection

Take a second to think about the following questions.

### 1. What's the difference between the pure LLM and the agent?
### 2. Why can the agent do math reliably while the pure LLM can't?
### 3. What other tools could you give an agent to make it more capable?
### 4. Can you think of a task where tool use is essential?

## Congratulations!

You've just built your first LLM agent! You've learned:

**LLMs predict text** - they don't compute  
**Agents = LLM + Tools** - extend capabilities beyond prediction  
**ReAct pattern** - Reason about what to do, then Act  
**Tool calling** - Agents can use external functions/APIs  
**Reliable execution** - Tools provide accuracy that prediction can't

## Next Time

We will learn to create a commitee of agents to solve complex tasks. 

We will learn how to ground language models using our data for more accurate query responses.